In [1]:
# preamble
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pandas as pd
import scipy.stats as stats
import scipy as sp
import statistics
import mat73
from scipy.sparse import spdiags, csr_matrix

In [2]:
class glm:
    def __init__(self, ST, P, hd):
        # remove nans and infinite values
        idx_finite = np.where(np.isfinite(P[:,1]))[0]
        idx_notnan = np.where(~np.isnan(P[:,1]))[0]
        keep_idx = np.intersect1d(idx_finite, idx_notnan)
        
        self.P = P[keep_idx,:]
        self.x = P[keep_idx,1]
        self.y = P[keep_idx,2]
        self.t = P[keep_idx,0]
        self.hd = (hd[keep_idx,0]*np.pi)/180; # 0-2pi
        self.dt = P[:,1]-P[:,0]
        self.ST = ST # spiketimes (not train)
        
    def get_size(self):
        '''get size of recording box'''
        
        boxsz = np.nanmax([np.nanmax(self.x), np.nanmax(self.y)])
        
        return boxsz
    
    
    
    def pos_map(self, nbins=10):
        '''design matrix for position variables'''
        
        boxsz = self.get_size()
        bins = np.arange(boxsz/nbins/2, boxsz-boxsz/nbins/2, round(boxsz/nbins))
        posgrid = np.zeros((len(self.x), nbins**2))
        
        for idx,val in enumerate(self.x):
            
            xvec = np.abs(self.x[idx]-bins); yvec = np.abs(self.y[idx]-bins);
            min_x = np.min(xvec)
            min_y = np.min(yvec)
            idx_x = np.where(xvec == min_x); idx_x = idx_x[0][0];
            idx_y = np.where(yvec == min_y); idx_y = idx_y[0][0];
            bin_idx = np.ravel_multi_index((idx_y,idx_x), dims=(nbins,nbins), order='C') # a11=0, a12=1, a13=2;
            posgrid[idx, bin_idx] = 1;
            
        return posgrid, bins
    
    
    
    def eb_map(self, nbins=10, rp=[75,75]):
        '''design matrix for egocentric variables'''
        
        refx = rp[0]; refy = rp[1];
        allo = np.arctan2(refy-self.y, refx-self.x) + (np.pi/2); # add 90 deg
        allo[allo<0] = allo[allo<0]+2*np.pi;
        ego = allo - self.hd; # shift from 0-2pi
        egogrid = np.zeros((len(self.P),nbins));
        bins = np.arange(2*np.pi/nbins/2, 2*np.pi-2*np.pi/nbins/2, 2*np.pi/nbins) # 10 bin ctrs
        
        for idx,val in enumerate(self.P):
            
            evec = np.abs(ego[idx]-bins)
            min_e = np.min(evec)
            idx_e = np.where(evec == min_e)
            egogrid[idx, idx_e] = 1;
            
        return egogrid, bins

### load data

In [3]:
filepath = 'sampleData.mat'
mat = scipy.io.loadmat(filepath)
ST = mat['ST']; P = mat['P']; HD = mat['hd']

### intialize class instance

In [13]:
g = glm(ST,P,HD)

## E+P model

### 1. calculate tuning curves (data)

In [150]:
HD = (HD + 180) % 360 - 180;
t = P[:,0]; tpf = t[1]-t[0]
x = P[:,1]
y = P[:,2]
boolean_spk = np.logical_and(t[0] <= ST, ST <= t[-1])
spikes = ST[boolean_spk == True]
edgesT = np.linspace(t[0], t[-1], len(t)+1)
binnedSpikes, timeEdges = np.histogram(spikes, edgesT)

# spatial bins
nBins = 10;
H, yedges, xedges = np.histogram2d(x, y, bins=10);
hitx = np.digitize(x, xedges)
hity= np.digitize(y, yedges)

# angular bin centers
num_Z_bins = 10; # 36 deg/bin
Z_bins = np.linspace(0,360,num_Z_bins+1)
Z_edges = np.linspace(-180,180,num_Z_bins+1)
Z_bin_ctrs = (((Z_bins[1]-Z_bins[0])/2) + Z_bins[0:-1])-180

# initialize structures
time_H = np.ones((10,10,10))*np.NaN
count_H = np.ones((10,10,10))*np.NaN
R_xyh = np.ones((10,10,10))*np.NaN
r_xyh = np.ones((10,10,10))*np.NaN
r_xy = np.ones((10,10))*np.NaN

## 3D tuning curve ##
for rr in range(nBins):
    for cc in range(nBins):
        
        # data from bin(rr,cc)
        idx = np.where(np.logical_and(rr == hitx,cc == hity))
        time_in_bin = len(idx)*tpf
        spk = binnedSpikes[idx]
        z_here = HD[idx]

        # HD occupancy
        zhist, _ = np.histogram(z_here, bins=Z_edges)
        z_idx = np.digitize(z_here, Z_edges)
        z_occ_here = zhist*tpf

        # criteria check
        bin_threshold = 0.4 # 400 ms/each
        rate_threshold = 0.5 # Hz
        bin_criteria = np.round(num_Z_bins*.5)
        num_bins_passed = np.sum(z_occ_here>bin_threshold)

        if num_bins_passed >= bin_criteria:
            for H in range(len(Z_bin_ctrs)):
                time_H[H,rr,cc] = z_occ_here[H]
                count_H[H,rr,cc] = zhist[H]
                idx_H = np.where(z_idx == H); idx_H = idx_H[0]

                if time_H[H,rr,cc] > rate_threshold:
                    spk_H = spk[idx_H]
                    r_xyh_here = np.sum(spk_H)/(count_H[H,rr,cc]*tpf)

                    # conditional ratemap
                    if np.isfinite(r_xyh_here):
                        r_xyh[H,rr,cc] = r_xyh_here

            # spatial ratemap
            r_xy[rr,cc] = np.nanmean(r_xyh[:,rr,cc])
            
            # conditional ratemap (zscore normalized)
            R_xyh[:,rr,cc] = stats.zscore(r_xyh[:,rr,cc], nan_policy='omit')

### 2. fit cosine to data

In [ ]:
rCutOff = .5
